In [3]:
import os
import numpy as np
import xarray as xr

In [31]:
exists_ok = lambda path: os.path.exists(path)

data_path = os.path.join(os.getcwd(), 'data')
zarr_path = os.path.join(data_path, 'sfv03_fcci.zarr')

print(f"{data_path} exists: {exists_ok(path=data_path)}")
print(f"{zarr_path} exists: {exists_ok(path=zarr_path)}")

def to_100(path_to_zarr:str, per_variable:bool=False):
	# define the max hectares value for 25 km data
	# remember: 
	# 	the target has been scaled by max_hectares_25 
	# 	before storing the dataset
	_max_hectares_25 = pow((111/4), 2)*100
	print(f"max hectares: {_max_hectares_25}")
	# open 25 km zarr dataset from path
	data = xr.open_zarr(path_to_zarr)
	# get variables
	variables = list(data.data_vars)
	# put the target variable as last in the list
	variables.append(variables.pop(variables.index('fcci_ba')))
	# coarsen data for each variable
	if per_variable:
		# create a numpy array of a selected shape to store the dataset
		dataset = np.ndarray(shape=(9, 966, 180, 360))
		for i, v in enumerate(variables):
			# get data for each variable and load it
			data_var = data[v].load()
			# if the variable is the target
			if v == 'fcci_ba':
				# multiply the coarsen data by the max_hectares value
				data_var = data_var * _max_hectares_25
			# aggregate with the coarsen function (25 to 100)
			data_var = data_var.coarsen(latitude=4, longitude=4, boundary='trim').mean()
			# put coarsened data to the new array
			dataset[i, ...] = data_var
		# store the new 100 km dataset as numpy
		np.save(os.path.join(data_path, "dataset_coarsen"), dataset)
	# use xarray functionalities to coarsen data
	else:
		# use xarray coarsen function
		zarr_100 = data.coarsen(latitude=4, longitude=4, boundary='trim').mean()
		# save the new dataset as zarr
		zarr_100.to_zarr(os.path.join(data_path, 'seasfire_v03_100km.zarr'))


/Users/edonno/VSCode/CMCC/ML4Fires/digital_twin_notebooks/data exists: False
/Users/edonno/VSCode/CMCC/ML4Fires/digital_twin_notebooks/data/sfv03_fcci.zarr exists: False


In [23]:
zarr_complete = xr.open_zarr(os.path.join(data_path, 'seasfire_v03_100km.zarr'))
zarr_complete

NameError: name 'data_path' is not defined

In [25]:
import xarray as xr
data = xr.open_zarr('../data/sfv03_fcci.zarr')
variables = list(data.data_vars)
print(variables)

variables.append(variables.pop(variables.index('fcci_ba')))
print(variables)

data = data[variables]
data

['fcci_ba', 'lai', 'lsm', 'lst_day', 'rel_hum', 'ssr', 'sst', 't2m_min', 'tp']
['lai', 'lsm', 'lst_day', 'rel_hum', 'ssr', 'sst', 't2m_min', 'tp', 'fcci_ba']


<xarray.Dataset>
Dimensions:    (time: 966, latitude: 720, longitude: 1440)
Coordinates:
  * latitude   (latitude) float64 89.88 89.62 89.38 ... -89.38 -89.62 -89.88
  * longitude  (longitude) float64 -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * time       (time) datetime64[ns] 2001-01-01 2001-01-09 ... 2021-12-27
Data variables:
    lai        (time, latitude, longitude) float32 ...
    lsm        (time, latitude, longitude) float32 ...
    lst_day    (time, latitude, longitude) float32 ...
    rel_hum    (time, latitude, longitude) float32 ...
    ssr        (time, latitude, longitude) float32 ...
    sst        (time, latitude, longitude) float32 ...
    t2m_min    (time, latitude, longitude) float32 ...
    tp         (time, latitude, longitude) float32 ...
    fcci_ba    (time, latitude, longitude) float32 ...
Attributes:
    crs:          EPSG:4326
    description:  The SeasFire Cube is a scientific datacube for seasonal fir...
    title:        SeasFire Cube: A Global Dataset for Seasonal Fire Modeling ...

In [28]:
data.fcci_ba

array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       ...,

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan

In [2]:
import sys, os

if os.path.dirname(os.getcwd()) not in sys.path:
	sys.path.append(os.path.dirname(os.getcwd()))

for i, p in enumerate(sys.path):
	print(f"{i} - {p}")

0 - /opt/anaconda3/envs/ml4fires/lib/python310.zip
1 - /opt/anaconda3/envs/ml4fires/lib/python3.10
2 - /opt/anaconda3/envs/ml4fires/lib/python3.10/lib-dynload
3 - 
4 - /opt/anaconda3/envs/ml4fires/lib/python3.10/site-packages
5 - /Users/edonno/VSCode/CMCC/ML4Fires


In [18]:
currnt_dir = os.getcwd()
parent_dir = os.path.dirname(currnt_dir)
name_dir = 'ML4Fires'

# if currnt_dir.split('/')[-1] !=
if os.path.dirname(os.getcwd()).split('/')[-1] == 'ML4Fires':
	print('sei in una subdir')

list_of_dir_names = os.getcwd().split('/')[1:]
for p in list_of_dir_names:
	print(list_of_dir_names.index(p), "\t ", p)

print(os.getcwd().split('/')[1:].index('ML4Fires'))
print(os.getcwd().split('ML4Fires')[-1].split('/'))

print(os.path.dirname(os.getcwd()).split('ML4Fires')[-1])
# os.path.dirname(os.getcwd())

sei in una subdir
0 	  Users
1 	  edonno
2 	  VSCode
3 	  CMCC
4 	  ML4Fires
5 	  digital_twin_notebooks
4
['', 'digital_twin_notebooks']



In [ ]:

from Fires._macros.macros import DRIVERS, TARGETS
